In [1]:
import pandas as pd

# Read 100000 rows for demo
df = pd.read_csv("Cleaned_data.csv", nrows=100000)

# double confirming for cleaning
df['Incident_zip'] = df['Incident_zip'].astype(str).str.strip()
df['Incident_zip'] = df['Incident_zip'].fillna('UNKNOWN')
df['Complaint_type'] = df['Complaint_type'].fillna('UNKNOWN')


print(df.head())


FileNotFoundError: [Errno 2] No such file or directory: 'Cleaned_data.csv'

In [ ]:

complaint_counts = df.groupby(['Incident_zip', 'Complaint_type']).size().reset_index(name='count')


print(complaint_counts.head())


  Incident_zip          Complaint_type  count
0        10000            DAMAGED TREE      1
1        10000         ILLEGAL PARKING      1
2        10000      NOISE - HELICOPTER      2
3        10000      SIDEWALK CONDITION      1
4        10000  STREET LIGHT CONDITION     10


 ### connect to neo4j

In [ ]:
!pip install py2neo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.2/177.2 kB 4.0 MB/s eta 0:00:00


In [ ]:
from py2neo import Graph, Node, Relationship

#connect to neo4j
graph = Graph("bolt://localhost:7689", auth=("neo4j", "12345678"))

# Import data and set the count field
def import_data_to_neo4j(df, graph):
    total_rows = len(df)

    # Used for batch data insertion
    for index, row in df.iterrows():
        zip_code = row['Incident_zip']
        complaint_type = row['Complaint_type']
        borough = row['Borough']

        # Retrieve the value of the count field
        count = row['count'] if 'count' in row else 1

        # Create Node
        zip_node = Node("Zip", code=zip_code)
        complaint_node = Node("ComplaintType", type=complaint_type)
        borough_node = Node("Borough", name=borough)

        # Use merge to make sure no dupulicate node
        graph.merge(zip_node, "Zip", "code")
        graph.merge(complaint_node, "ComplaintType", "type")
        graph.merge(borough_node, "Borough", "name")

        # createe count field
        has_complaint = Relationship(zip_node, "HAS_COMPLAINT", complaint_node)
        has_complaint['count'] = count
        located_in = Relationship(zip_node, "LOCATED_IN", borough_node)

        # insert relationshsip
        graph.create(zip_node)
        graph.create(complaint_node)
        graph.create(borough_node)
        graph.create(has_complaint)
        graph.create(located_in)


        if (index + 1) % 1000 == 0:
            print(f"Progress: {index + 1}/{total_rows} rows imported.")

# import data
import_data_to_neo4j(df, graph)

print("Data import completed!")


Progress: 1000/100000 rows imported.
Progress: 2000/100000 rows imported.
Progress: 3000/100000 rows imported.
Progress: 4000/100000 rows imported.
Progress: 5000/100000 rows imported.
Progress: 6000/100000 rows imported.
Progress: 7000/100000 rows imported.
Progress: 8000/100000 rows imported.
Progress: 9000/100000 rows imported.
Progress: 10000/100000 rows imported.
Progress: 11000/100000 rows imported.
Progress: 12000/100000 rows imported.
Progress: 13000/100000 rows imported.
Progress: 14000/100000 rows imported.
Progress: 15000/100000 rows imported.
Progress: 16000/100000 rows imported.
Progress: 17000/100000 rows imported.
Progress: 18000/100000 rows imported.
Progress: 19000/100000 rows imported.
Progress: 20000/100000 rows imported.
Progress: 21000/100000 rows imported.
Progress: 22000/100000 rows imported.
Progress: 23000/100000 rows imported.
Progress: 24000/100000 rows imported.
Progress: 25000/100000 rows imported.
Progress: 26000/100000 rows imported.
Progress: 27000/10000

In [ ]:
from py2neo import Graph

#connect to neo4j
graph = Graph("bolt://localhost:7689", auth=("neo4j", "12345678"))

def update_count_in_neo4j(complaint_counts, graph):
    for index, row in complaint_counts.iterrows():
        zip_code = row['Incident_zip']
        complaint_type = row['Complaint_type']
        count = row['count']

        #update count field
        graph.run("""
        MATCH (z:Zip {code: $zip_code})-[r:HAS_COMPLAINT]->(c:ComplaintType {type: $complaint_type})
        SET r.count = $count
        """, zip_code=zip_code, complaint_type=complaint_type, count=count)

    print("Count fields updated successfully!")


update_count_in_neo4j(complaint_counts, graph)

Count fields updated successfully!


In [ ]:
# check if count is updated correctly
def check_count_in_neo4j(graph):
    query = """
    MATCH (z:Zip)-[r:HAS_COMPLAINT]->(c:ComplaintType)
    RETURN z.code AS zip_code, c.type AS complaint_type, r.count AS count
    LIMIT 10;
    """

    result = graph.run(query).data()

    print("Results:")
    for row in result:
        print(f"ZIP Code: {row['zip_code']}, Complaint Type: {row['complaint_type']}, Count: {row['count']}")


check_count_in_neo4j(graph)

Results:
ZIP Code: 10305, Complaint Type: HEAT/HOT WATER, Count: 28
ZIP Code: 11223, Complaint Type: HEAT/HOT WATER, Count: 131
ZIP Code: 11102, Complaint Type: HEAT/HOT WATER, Count: 86
ZIP Code: 10309, Complaint Type: HEAT/HOT WATER, Count: 1
ZIP Code: 11105, Complaint Type: HEAT/HOT WATER, Count: 49
ZIP Code: 10031, Complaint Type: HEAT/HOT WATER, Count: 458
ZIP Code: 10016, Complaint Type: HEAT/HOT WATER, Count: 49
ZIP Code: 11379, Complaint Type: HEAT/HOT WATER, Count: 24
ZIP Code: 11365, Complaint Type: HEAT/HOT WATER, Count: 16
ZIP Code: 11354, Complaint Type: HEAT/HOT WATER, Count: 48


query

In [ ]:
#Define the query: Query the top 5 complaint types for a given ZIP code
def get_top_complaint_types_for_zip(zip_code):
    query = """
    MATCH (z:Zip {code: $zip_code})-[r:HAS_COMPLAINT]->(c:ComplaintType)
    RETURN c.type AS complaint_type, SUM(r.count) AS total_complaints
    ORDER BY total_complaints DESC
    LIMIT 5;
    """
    # Execute the query and return the results
    result = graph.run(query, zip_code=zip_code).data()

    # print result
    for row in result:
        print(f"Complaint Type: {row['complaint_type']}, Complaints: {row['total_complaints']}")

    return result

In [ ]:
# Example: Query the complaint types for ZIP code 10001
zip_code = "10001"
get_top_complaint_types_for_zip(zip_code)

Complaint Type: PAINT/PLASTER, Complaints: 1762
Complaint Type: ABANDONED VEHICLE, Complaints: 1532
Complaint Type: WATER LEAK, Complaints: 1304
Complaint Type: DERELICT VEHICLES, Complaints: 1020
Complaint Type: ELECTRIC, Complaints: 917


[{'complaint_type': 'PAINT/PLASTER', 'total_complaints': 1762},
 {'complaint_type': 'ABANDONED VEHICLE', 'total_complaints': 1532},
 {'complaint_type': 'WATER LEAK', 'total_complaints': 1304},
 {'complaint_type': 'DERELICT VEHICLES', 'total_complaints': 1020},
 {'complaint_type': 'ELECTRIC', 'total_complaints': 917}]

In [ ]:
def get_top_and_least_complaints(zip_code):
    # query most common complaint type (top5)
    query_top = """
    MATCH (z:Zip {code: $zip_code})-[r:HAS_COMPLAINT]->(c:ComplaintType)
    WITH c, SUM(r.count) AS total_complaints
    ORDER BY total_complaints DESC
    LIMIT 5
    RETURN c.type AS complaint_type, total_complaints AS complaints;
    """

    # query least common complaint type (top5)
    query_least = """
    MATCH (z:Zip {code: $zip_code})-[r:HAS_COMPLAINT]->(c:ComplaintType)
    WITH c, SUM(r.count) AS total_complaints
    ORDER BY total_complaints ASC
    LIMIT 5
    RETURN c.type AS complaint_type, total_complaints AS complaints;
    """

    # Execute the query and return the results
    result_top = graph.run(query_top, zip_code=zip_code).data()
    result_least = graph.run(query_least, zip_code=zip_code).data()

    # print result
    print("Top 5 Complaints:")
    for row in result_top:
        print(f"Complaint Type: {row['complaint_type']}, Complaints: {row['complaints']}")

    print("\nLeast 5 Complaints:")
    for row in result_least:
        print(f"Complaint Type: {row['complaint_type']}, Complaints: {row['complaints']}")

    return result_top, result_least

# Example: Query the complaint types for ZIP code 11101
zip_code = '11101'
get_top_and_least_complaints(zip_code)

Top 5 Complaints:
Complaint Type: NOISE - HELICOPTER, Complaints: 626
Complaint Type: SAFETY, Complaints: 469
Complaint Type: ELECTRONICS WASTE APPOINTMENT, Complaints: 276
Complaint Type: DEAD ANIMAL, Complaints: 217
Complaint Type: RESIDENTIAL DISPOSAL COMPLAINT, Complaints: 200

Least 5 Complaints:
Complaint Type: ENCAMPMENT, Complaints: 1
Complaint Type: FOOD ESTABLISHMENT, Complaints: 1
Complaint Type: NON-EMERGENCY POLICE MATTER, Complaints: 1
Complaint Type: SMOKING, Complaints: 1
Complaint Type: STREET SIGN - DAMAGED, Complaints: 1


([{'complaint_type': 'NOISE - HELICOPTER', 'complaints': 626},
  {'complaint_type': 'SAFETY', 'complaints': 469},
  {'complaint_type': 'ELECTRONICS WASTE APPOINTMENT', 'complaints': 276},
  {'complaint_type': 'DEAD ANIMAL', 'complaints': 217},
  {'complaint_type': 'RESIDENTIAL DISPOSAL COMPLAINT', 'complaints': 200}],
 [{'complaint_type': 'ENCAMPMENT', 'complaints': 1},
  {'complaint_type': 'FOOD ESTABLISHMENT', 'complaints': 1},
  {'complaint_type': 'NON-EMERGENCY POLICE MATTER', 'complaints': 1},
  {'complaint_type': 'SMOKING', 'complaints': 1},
  {'complaint_type': 'STREET SIGN - DAMAGED', 'complaints': 1}])

In [ ]:
def get_top_and_least_zip_for_complaint(complaint_type, graph):
    # query zip code with most complaint about (e.g NOISE - STREET/SIDEWALK )
    query_top = """
    MATCH (z:Zip)-[r:HAS_COMPLAINT]->(c:ComplaintType {type: $complaint_type})
    RETURN z.code AS zip_code, SUM(r.count) AS total_complaints
    ORDER BY total_complaints DESC
    LIMIT 1;
    """

    # query zip code with least complaint about (e.g NOISE - STREET/SIDEWALK )
    query_least = """
    MATCH (z:Zip)-[r:HAS_COMPLAINT]->(c:ComplaintType {type: $complaint_type})
    RETURN z.code AS zip_code, SUM(r.count) AS total_complaints
    ORDER BY total_complaints ASC
    LIMIT 1;
    """

    # Execute the query and return the results
    result_top = graph.run(query_top, complaint_type=complaint_type).data()
    result_least = graph.run(query_least, complaint_type=complaint_type).data()

    # print result
    if result_top:
        print(f"Most Complaints for Complaint Type: {complaint_type}")
        print(f"ZIP Code: {result_top[0]['zip_code']}, Complaints: {result_top[0]['total_complaints']}")

    if result_least:
        print(f"\nLeast Complaints for Complaint Type: {complaint_type}")
        print(f"ZIP Code: {result_least[0]['zip_code']}, Complaints: {result_least[0]['total_complaints']}")

    return result_top, result_least



In [ ]:
# use NOISE - STREET/SIDEWALK as example
complaint_type = "NOISE - STREET/SIDEWALK"
get_top_and_least_zip_for_complaint(complaint_type, graph)

Most Complaints for Complaint Type: NOISE - STREET/SIDEWALK
ZIP Code: 10305, Complaints: 1466

Least Complaints for Complaint Type: NOISE - STREET/SIDEWALK
ZIP Code: 11379, Complaints: 1


([{'zip_code': '10305', 'total_complaints': 1466}],
 [{'zip_code': '11379', 'total_complaints': 1}])